In [ ]:
import pandas as pd

In [ ]:
import json

def parse_sarcasm_json_dataset(file):
    for line in open(file, 'r'):
        yield json.loads(line)

In [ ]:
data = list(parse_sarcasm_json_dataset('../../data/news_headlines_for_sarcasm_detection/Sarcasm_Headlines_Dataset_v2.json'))

df = pd.DataFrame(data)
df

In [ ]:
dataset_list = df["headline"].values[:40000].tolist()

In [ ]:
len(max(dataset_list, key=len))

In [ ]:
from transformers import pipeline

translation_checkpoint = "sdadas/flan-t5-base-translator-en-pl"

generator = pipeline("translation_en_to_pl", model=translation_checkpoint)

In [ ]:
import os.path

def add_to_file(index, translated_list):
    if os.path.isfile('../../data/translated/sarcasm/sarcasm_headlines_dataset_pl.json') == True:
        data_df = pd.read_json('../../data/translated/sarcasm/sarcasm_headlines_dataset_pl.json')
        translated_before_list = data_df["headline_pl"].values.tolist()
        linked_list = translated_before_list + translated_list
    else:
        linked_list = translated_list
    new_df = df[:index].copy(deep=True)
    new_df['headline_pl'] = linked_list
    new_df.to_json(r'../../data/translated/sarcasm/sarcasm_headlines_dataset_pl.json')

In [ ]:
translated_list = []
for index, sentence in enumerate(dataset_list):
  if index % 250 == 0:
      add_to_file(index, translated_list)
      translated_list = []
  print(index)
  translated = generator(sentence, max_length=1500)
  translated_text = translated[0]["translation_text"]
  translated_list.append(translated_text)

In [ ]:
dataset_list[:5]

In [ ]:
translated_list[:5]

In [ ]:

data_df = pd.read_json('../../data/translated/sarcasm/sarcasm_headlines_dataset_pl.json')
translated_texts_number = len(data_df["headline_pl"].values.tolist()) + len(translated_list)

add_to_file(translated_texts_number, translated_list)

In [ ]:
data_df = pd.read_json('../../data/translated/sarcasm/sarcasm_headlines_dataset_pl.json')
data_df